# The Utility code
## This code helps in analysing the scenarios record and display the required results.

### CARLA Location and Connection information
>#### Note: The Jupyter notebook should be run from the mlmas environment.
>#### Make sure that the CARLA simulator is running.

In [1]:
# Specify the carla location and IP to connect with.
import os

prj_dir = os.getcwd().replace('/MLMAS_Framework/tools','')

CARLA_PATH = prj_dir+'/CARLA_0.9.10.1'


HOST = '127.0.0.1'
PORT = 2000



### Import the required libraries

In [2]:
# run this code and make sure CARLA simulator is running

import glob

import sys
import pandas as pd
import json

try:
    sys.path.append(glob.glob(CARLA_PATH+'/PythonAPI/carla/dist/carla-0.9.10-py3.7-linux-x86_64.egg')[0])
except IndexError:
    print("error >>>")

import carla



## 1. Configuration
### Record file path to analyse

In [17]:
recorder_filename = prj_dir+'/results/records/tfus/RouteScenario_8_rep0.log'
recorder_filename2 = prj_dir+'/results/records/MLMAS_tfus/RouteScenario_8_rep0.log'


## 2. Analysing the Record Logs

### - List of the possible blocked agents cases in this scenario:

In [4]:
TIME = 30
DISTANCE = 10
try:

    client = carla.Client(HOST, PORT)
    client.set_timeout(60.0)

    print(client.show_recorder_actors_blocked(recorder_filename, TIME, DISTANCE))

finally:
    pass


Version: 1
Map: Town06
Date: 10/28/22 14:06:56

    Time     Id Actor                                 Duration
       0    193 walker.pedestrian.0007                     805
       0    188 walker.pedestrian.0010                     805
       0    186 walker.pedestrian.0007                     805
     588    242 vehicle.carlamotors.carlacola              217
     592    197 vehicle.diamondback.century                213
     626    182 vehicle.lincoln.mkz2017                    179
     633    227 vehicle.lincoln.mkz2017                    103
     635    320 vehicle.lincoln.mkz2017                    102
     626    303 vehicle.audi.a2                            100
     690    317 vehicle.audi.etron                          90
     665    309 vehicle.diamondback.century                 90
     626    316 vehicle.tesla.model3                        90
     598    305 vehicle.gazelle.omafiets                    90
     691    294 vehicle.jeep.wrangler_rubicon               90
     70

### - List of all collission cases in this scenario

In [18]:
try:

    client = carla.Client(HOST, PORT)
    client.set_timeout(60.0)

    print(client.show_recorder_collisions(recorder_filename, "v", "a"))

finally:
    pass

Version: 1
Map: Town02
Date: 10/28/22 18:20:11

    Time  Types     Id Actor 1                                 Id Actor 2                            
     155   v v     681 vehicle.lincoln.mkz2017                784 vehicle.tesla.model3               
     156   v v     681 vehicle.lincoln.mkz2017                784 vehicle.tesla.model3               
     212   v v     681 vehicle.lincoln.mkz2017                784 vehicle.tesla.model3               
     269   v v     681 vehicle.lincoln.mkz2017                784 vehicle.tesla.model3               
     326   v v     681 vehicle.lincoln.mkz2017                784 vehicle.tesla.model3               
     382   v v     681 vehicle.lincoln.mkz2017                784 vehicle.tesla.model3               

Frames: 7985
Duration: 399 seconds



## 3. Run the Scenario in a Specific Period
### - Baseed on the above code information specify the following:
in the replay_file method replay_file(record_filename, start_sec, end_sec, camera_actor_id):
- start_sec: From which second you want to start the scenario (eg. you want to start 3 sec before the collision happen based on the previous code information)
- duration: Which second you want the scenario to stop.
- camera_actor_id: based on the previous code information, you can get the id of the vehicle you want to follow, and specify it in this argument. however, if the id is invalud, the camera will be always shows from top. (By default configured to get the id of the hero-car that controlled by the model)

In [19]:
## Change the start time and the duration for the scenario to rerun in the specify period.
Start_Time = 154
Duration = 3000
p = "2"
def get_hero_vehicle(client):
    try:
        act_list = client.show_recorder_file_info(recorder_filename, False).splitlines()
        
        for i in range(len(act_list)):
                if "role_name = hero" in act_list[i]:
                    s = act_list[i-5]
                    s = s[7:s.find(':')]
                    return int(s)

    finally:
        pass
try:
    
    client = carla.Client(HOST, PORT)
    client.set_timeout(60.0)
    client.reload_world()
    client.replay_file(recorder_filename,
                       time_start=Start_Time,
                       duration=Duration,
                       follow_id = get_hero_vehicle(client))

finally:
    pass


In [25]:
## Change the start time and the duration for the scenario to rerun in the specify period.
Start_Time = 230
Duration = 3000
p = "2"
def get_hero_vehicle(client):
    try:
        act_list = client.show_recorder_file_info(recorder_filename2, False).splitlines()
        
        for i in range(len(act_list)):
                if "role_name = hero" in act_list[i]:
                    s = act_list[i-5]
                    s = s[7:s.find(':')]
                    return int(s)

    finally:
        pass
try:
    
    client = carla.Client(HOST, PORT)
    client.set_timeout(60.0)
    client.reload_world()
    client.replay_file(recorder_filename2,
                       time_start=Start_Time,
                       duration=Duration,
                       follow_id = get_hero_vehicle(client))

finally:
    pass


### =================== End Of Scenario Runner ===================

# Results Statistics

## 1. configuration
### Configure of the results files location

In [ ]:
mlmas_leaderboard_results_file = os.getcwd().replace('/MLMAS_Framework/tools','/results/MLMAS_tfus_results.json')
mlmas_additional_metrics_file = os.getcwd().replace('/MLMAS_Framework/tools','/results/MLMAS_tfus_results_jason_metrics.csv')
lav_leaderboard_results = os.getcwd().replace('/MLMAS_Framework/tools','/results/transfuser_longest6.json')


### Load the files

In [ ]:
def loadLeaderboardJson(filepath):
    f = open(os.path.realpath(filepath))
    return json.load(f)
    

def geLeaderboardMetrics(jsn):
    leaderboard_metrics = jsn['values']
    leaderboard_df = pd.DataFrame([jsn['values']],
                                  columns=jsn['labels'])
    leaderboard_df = leaderboard_df.astype(float)
    
    return leaderboard_df



mlmas_leaderboard_jsn = loadLeaderboardJson(mlmas_leaderboard_results_file)
lav_leaderboard_jsn = loadLeaderboardJson(lav_leaderboard_results)


## 2. Display the leaderboard results

In [ ]:
mlmas_leaderboard_df = geLeaderboardMetrics(mlmas_leaderboard_jsn)
lav_leaderboard_df = geLeaderboardMetrics(lav_leaderboard_jsn)
display(mlmas_leaderboard_df)
print("== MLMAS Framework with LAV Model Leaderboard Metrics results ==")
print("== ========================================================== ==")
display(lav_leaderboard_df)
print("== LAV Model Only Leaderboard Metrics results ==")

In [ ]:
def mergeTwoLeaderbordMetricsResults(title1, title2, df1, df2):
    score_columns = df1.columns.values[0:3]
    df_s1 = df1[score_columns]
    df_s1.loc[:,2:3] = df_s1[score_columns[2]].values*100
    df_s2 = df2[score_columns]
    df_s2.loc[:,2:3] = df_s2[score_columns[2]].values*100
    
    collisions_columns = df1.columns.values[3:6]
    
    road_columns = df1.columns.values[6:9]
    
    other_columns = df1.columns.values[9:]
    
    final_df_s = pd.DataFrame({title1: df_s1.values[0]
                  , title2: df_s2.values[0]
                  , 'Results_Higher_is_Better': score_columns}) 
    
    final_df_c = pd.DataFrame({title1: df1[collisions_columns].values[0]
          , title2: df2[collisions_columns].values[0]
          , 'Results_Lower_is_Better': collisions_columns}) 
    
    final_df_r = pd.DataFrame({title1: df1[road_columns].values[0]
              , title2: df2[road_columns].values[0]
              , 'Results_Lower_is_Better': road_columns}) 
    
    final_df_o = pd.DataFrame({title1: df1[other_columns].values[0]
          , title2: df2[other_columns].values[0]
          , 'Results_Lower_is_Better': other_columns}) 
        
        
    final_df_s.index = final_df_s.Results_Higher_is_Better
    final_df_c.index = final_df_c.Results_Lower_is_Better
    final_df_o.index = final_df_o.Results_Lower_is_Better
    final_df_r.index = final_df_r.Results_Lower_is_Better
    return final_df_s, final_df_c, final_df_o, final_df_r

def show_values(g, pr="%", is_vertical = True):
    for p in g.patches:
        width = p.get_width()
        height = p.get_height()
        x, y = p.get_xy() 
        if is_vertical:
            g.annotate(f'{round(height,1)}{pr}' if pr == "%" else f'{height}{pr}', (x + width/2, y + height*1.02), ha='center')
        else:
            g.annotate(f'{width}{pr}', (x + width + 0.09, y + height/2), ha='center')



In [ ]:
import matplotlib.pyplot as plt
font = {'family' : 'DejaVu Sans',
        'weight' : 'bold',
        'size'   : 17}

plt.rc('font', **font)


final_df_s, final_df_c, final_df_o, final_df_r = mergeTwoLeaderbordMetricsResults('transfuser','ML-MAS', lav_leaderboard_df ,mlmas_leaderboard_df)
graph = final_df_s.plot.bar(figsize=(13,8))

graph2 = final_df_c.plot.bar(figsize=(13,8))
graph3 = final_df_r.plot.bar(figsize=(13,8))
graph4 = final_df_o.plot.bar(figsize=(11,9))

plt.tight_layout()
show_values(graph)
show_values(graph2, pr="")
show_values(graph3, pr="")
show_values(graph4, pr="")

## 3. Jason Agent Additional Metrics

In [ ]:
jason_metrics_csv = pd.read_csv(mlmas_additional_metrics_file, delimiter=';')

final_columns = jason_metrics_csv.columns.values
final_columns[6], final_columns[7] = final_columns[7], final_columns[6]

jason_metrics_csv = jason_metrics_csv[final_columns]
additional_metrics_columns = ["Total Frames", 
                              "Front \nCollision Avoidance",
                             "Far Crossing \nCollision Avoidance",
                             "Close Crossing \nCollision Avoidance",
                             "Back \nCollision Avoidance",
                             "Traffic Light \nGreen [Go]",
                             "Traffic Light \nSlowdown",
                             "Traffic jam \ninterference"]
jason_metrics_csv.columns = additional_metrics_columns

print("== The additional Jason Agent metrics in each scenario")
jason_metrics_csv

## Summary of the additional metrics

In [ ]:
jason_metrics_csv.describe()

In [ ]:
font = {'family' : 'DejaVu Sans',
        'weight' : 'bold',
        'size'   : 17}

plt.rc('font', **font)

mean_percentage = round(jason_metrics_csv.describe()*100,1)
mean_percentage = mean_percentage[additional_metrics_columns[1:]][1:2].values[0]
final_df_s = pd.DataFrame({'Interference_Percentage': mean_percentage
                  , 'Interference_Type': additional_metrics_columns[1:]}) 

final_df_s.index = final_df_s.Interference_Type
final_df_s = final_df_s.sort_values(by=["Interference_Percentage"])
graph = final_df_s.plot.barh(figsize=(13,13))
graph.set_title("Average Jason Agent Interference Percentage of the Whole Evaluation Time")
show_values(graph, is_vertical= False)

print("Total Average Agent Interferance Percentage: (%.1f%s)"%(mean_percentage.sum(),"%"))

In [ ]:

total_df = pd.DataFrame({"Models":["ML-Model","(BDI) Agent"], "Contribution": [100 - mean_percentage.sum(), mean_percentage.sum()]})
total_df.index = total_df.Models

plot = total_df.plot.pie(y='Contribution', title="The ML-Model vs (BDI) Agent Contribution", legend=False, \
                   autopct='%1.1f%%', explode=(0, 0.1), \
                   shadow=True, 
                         startangle=-4,figsize=(8, 8))
